In [38]:
import pandas as pd

pd.options.display.float_format = "{:,.2f}".format

In [39]:
inventory = pd.DataFrame(
    {
        "id": [1, 2, 3, 4, 5],
        "name": ["black pen", "stapler", "ruler", "tissues", "eraser"],
        "wholesale price": [0.25, 1.5, 1.15, 1.0, 0.3],
        "retail price": [1.75, 5.99, 2.0, 2.5, 1.5],
        "sales": [251, 37, 20, 302, 24],
    }
)
inventory


,id,name,wholesale price,retail price,sales
0,1,black pen,0.25,1.75,251
1,2,stapler,1.50,5.99,37
2,3,ruler,1.15,2.00,20
3,4,tissues,1.00,2.50,302
4,5,eraser,0.30,1.50,24


In [40]:
# Add net profit and 15%, 20% and 25% sales tax columns to the dataframe
inventory["net_profit"] = (
    inventory["retail price"] - inventory["wholesale price"]
) * inventory["sales"]
inventory["tax_15"] = inventory["net_profit"] * 0.85
inventory["tax_20"] = inventory["net_profit"] * 0.8
inventory["tax_25"] = inventory["net_profit"] * 0.75
inventory

,id,name,wholesale price,retail price,sales,net_profit,tax_15,tax_20,tax_25
0,1,black pen,0.25,1.75,251,376.50,320.02,301.20,282.38
1,2,stapler,1.50,5.99,37,166.13,141.21,132.90,124.60
2,3,ruler,1.15,2.00,20,17.00,14.45,13.60,12.75
3,4,tissues,1.00,2.50,302,453.00,385.05,362.40,339.75
4,5,eraser,0.30,1.50,24,28.80,24.48,23.04,21.60


In [41]:
# Show the profit reduction as a series with each tax application
(
    (inventory["net_profit"].sum() - inventory[["tax_15", "tax_20", "tax_25"]].sum())
    .rename("Profit reduction")
    .apply(lambda x: f"${x:.2f}")
)

tax_15    $156.21
tax_20    $208.29
tax_25    $260.36
Name: Profit reduction, dtype: object

# Using the `.assign` method
This lets you chain methods inside the parens of the assign statement.

In [42]:
inventory = pd.DataFrame(
    {
        "id": [1, 2, 3, 4, 5],
        "name": ["black pen", "stapler", "ruler", "tissues", "eraser"],
        "wholesale price": [0.25, 1.5, 1.15, 1.0, 0.3],
        "retail price": [1.75, 5.99, 2.0, 2.5, 1.5],
        "sales": [251, 37, 20, 302, 24],
    }
)
# `.assign` returns a new dataframe rather than modifying the current one
inventory = inventory.assign(
    net_profit=(inventory["retail price"] - inventory["wholesale price"])
    * inventory["sales"]
)
inventory = inventory.assign(
    tax_15=inventory["net_profit"] * 0.85,
    tax_20=inventory["net_profit"] * 0.8,
    tax_25=inventory["net_profit"] * 0.75,
)
inventory

,id,name,wholesale price,retail price,sales,net_profit,tax_15,tax_20,tax_25
0,1,black pen,0.25,1.75,251,376.50,320.02,301.20,282.38
1,2,stapler,1.50,5.99,37,166.13,141.21,132.90,124.60
2,3,ruler,1.15,2.00,20,17.00,14.45,13.60,12.75
3,4,tissues,1.00,2.50,302,453.00,385.05,362.40,339.75
4,5,eraser,0.30,1.50,24,28.80,24.48,23.04,21.60


# Extension questions
1. An alternative tax plan would charge a 25% tax but only on products from which you would net more than 20000. How much profit would you make now?
2. Yet another alternative tax plan would charge a 25% tax on products whose retail price is greater than 80, a 10% tax on products whose retail price is between 30 and 80, and no tax on other products.
3. These long floating point numbers are getting hard to read. Set the `float_format` option in pandas such that floating point numbers will be displayed with commas every three digits before the decimal point and only two digits after the decimal point.

In [43]:
inventory = pd.DataFrame(
    {
        "id": [1, 2, 3, 4, 5, 6, 7],
        "name": [
            "black pen",
            "stapler",
            "ruler",
            "tissues",
            "eraser",
            "office chair",
            "foot rest",
        ],
        "wholesale price": [0.25, 1.5, 1.15, 1.0, 0.3, 43.6, 12],
        "retail price": [1.75, 5.99, 2.0, 2.5, 1.5, 170, 59],
        "sales": [22351, 375, 520, 38902, 242, 93, 41],
    }
)
# `.assign` returns a new dataframe rather than modifying the current one
inventory = inventory.assign(
    net_profit=(inventory["retail price"] - inventory["wholesale price"])
    * inventory["sales"]
)
inventory

,id,name,wholesale price,retail price,sales,net_profit
0,1,black pen,0.25,1.75,22351,"33,526.50"
1,2,stapler,1.50,5.99,375,"1,683.75"
2,3,ruler,1.15,2.00,520,442.00
3,4,tissues,1.00,2.50,38902,"58,353.00"
4,5,eraser,0.30,1.50,242,290.40
5,6,office chair,43.60,170.00,93,"11,755.20"
6,7,foot rest,12.00,59.00,41,"1,927.00"


In [44]:
# 1. 25% tax on items that net over 20,000
# Method A - final calculation
# Since this amount would change depending on the amount sold, the calculation
# method (A) would be superior than (B)
print(
    (inventory["net_profit"][inventory["net_profit"] > 20000] * 0.75).sum()
    + inventory["net_profit"][inventory["net_profit"] <= 20000].sum()
)
# Method B - new series
ext1 = inventory.assign(
    effective_profit=inventory.apply(
        lambda row: row["net_profit"] * (1 if row["net_profit"] <= 20000 else 0.75),
        axis=1,
    )
)
ext1["effective_profit"].sum()

85007.975


np.float64(85007.97499999999)

In [45]:
# 2. Apply 25% of goods whose retail price is > 80, 10% on 30 - 80, and none on others
#    This would be best modifying the dataframe columns and using that in calculating
#    profits
def calculate_tax_rate(row):
    retail = row["retail price"]
    if retail > 80:
        tax_rate = 0.25
    elif retail > 30:
        tax_rate = 0.1
    else:
        tax_rate = 0
    return tax_rate


ext2 = inventory.assign(tax_rate=inventory.apply(calculate_tax_rate, axis=1))

# now calculate the net profit
net = (
    (ext2["retail price"] - ext2["wholesale price"])
    * ext2["sales"]
    * (1 - ext2["tax_rate"])
).sum()
print(f"Net profit: ${net:.2f}")

Net profit: $104846.35


In [46]:
# 3. Set the `float_format` option in pandas such that floating point numbers will be displayed with commas every three digits before the decimal point and only two digits after the decimal point.